In [ ]:
import pandas as pd
import yellowbrick as yb
import os
from yellowbrick.classifier import ConfusionMatrix
from sklearn.linear_model import LogisticRegression
import pandas
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVR
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from yellowbrick.classifier import ROCAUC
from sklearn.model_selection import train_test_split
import seaborn as sns
from imblearn.over_sampling import SMOTE
import warnings
from sklearn.model_selection import TimeSeriesSplit
from sklearn.naive_bayes import GaussianNB
from yellowbrick.model_selection import FeatureImportances
from yellowbrick.classifier import ClassificationReport
warnings.filterwarnings('ignore')
%matplotlib inline
gnb = GaussianNB()
knn = KNeighborsClassifier()

In [ ]:
#Display all columns 
pd.set_option('display.max_columns', None)

In [ ]:
#View first 5 instances
df = pd.read_csv('higher_ed_summary.csv')
df.head()

In [ ]:
#Number of closed schools
df['closed_ind'].sum()

In [ ]:
df.shape

In [ ]:
#plot different features v closed_ind
%matplotlib inline

In [ ]:
#visualize between the relationship between the features and target using scatterplots
sns.pairplot(df, x_vars=['avg_percent_ft_ug_aid','slope_grad_enrollment','sector'], y_vars='closed_ind', height =4, aspect=0.7, kind='reg')

In [ ]:
df.head()

In [ ]:
#Drop instances with less than 300 features.
df1 = df.dropna(axis=0, thresh=300)
df1.head()

In [ ]:
df1.shape

In [ ]:
#Drop features wil null values.
df2 = df1.dropna(axis=1)
df2.head()

In [ ]:
df2.shape

In [ ]:
#Pick target feature
X = df2.loc[:, ~df2.columns.isin(['closed_ind','instname','unitid'])]
y = df2.loc[:, 'closed_ind']

In [ ]:
#Maps all feature values to the interval (0,1)
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

# Evaluate the models using k-fold cross-validation

In [ ]:
from yellowbrick.target import ClassBalance

X = df2.loc[:, ~df2.columns.isin(['closed_ind','instname','unitid'])]
y = df2.loc[:, 'closed_ind']

# Instantiate the visualizer
visualizer = ClassBalance(labels=["open", "closed"])

visualizer.fit(y)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
#Data imbalance in our dataset
y.value_counts()

In [ ]:
#Import SMOTE to address class imbalance
smt = SMOTE()
X, y = smt.fit_sample(X, y)

In [ ]:
# Instantiate the visualizer
visualizer = ClassBalance(labels=["open", "closed"])

visualizer.fit(y)        # Fit the data to the visualizer
visualizer.show()        # Finalize and render the figure

In [ ]:
y.value_counts()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Logistic Regression

In [ ]:
cross_val_score(LogisticRegression(solver='liblinear',multi_class='ovr'), X, y,cv=5)

In [ ]:
cross_val_predict(LogisticRegression(solver='liblinear',multi_class='ovr'), X, y,cv=5)

In [ ]:

viz = ConfusionMatrix(LogisticRegression())
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.show()

In [ ]:
X = df2.loc[:, ~df2.columns.isin(['closed_ind','instname','unitid'])]
y = df2.loc[:, 'closed_ind']
X_train, X_test, y_train, y_test = train_test_split(X, y)
oz = ROCAUC(LogisticRegression())
oz.fit(X_train, y_train)
oz.score(X_test, y_test)
oz.show()

In [ ]:
X = df2.loc[:, ~df2.columns.isin(['closed_ind','instname','unitid'])]
y = df2.loc[:, 'closed_ind']

# Label the target classes
classes = ["open", "closed"]

# Create the training and test data
tscv = TimeSeriesSplit()
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Instantiate the classification model and visualizer
model = LogisticRegression()
visualizer = ClassificationReport(model, classes=classes, support=True)

visualizer.fit(X_train, y_train)        # Fit the visualizer and the model
visualizer.score(X_test, y_test)        # Evaluate the model on the test data
visualizer.show()                       # Finalize and show the figure

# Support Vector Machine

In [ ]:
cross_val_score(SVC(gamma='auto'), X, y,cv=10)


In [ ]:
cross_val_predict(SVC(gamma='auto'), X, y,cv=10)

In [ ]:
viz = ConfusionMatrix(SVC())
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.show()

In [ ]:
X = df2.loc[:, ~df2.columns.isin(['closed_ind','instname','unitid'])]
y = df2.loc[:, 'closed_ind']

# Specify the target classes
classes = ["open", "closed"]

# Create the training and test data
tscv = TimeSeriesSplit()
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Instantiate the classification model and visualizer
model = SVC()
visualizer = ClassificationReport(model, classes=classes, support=True)

visualizer.fit(X_train, y_train)        # Fit the visualizer and the model
visualizer.score(X_test, y_test)        # Evaluate the model on the test data
visualizer.show()                       # Finalize and show the figure

# Random Forest Classifier 

In [ ]:
cross_val_score(RandomForestClassifier(n_estimators=30), X, y,cv=10)

In [ ]:
cross_val_predict(RandomForestClassifier(n_estimators=30), X, y,cv=10)

In [ ]:
viz = ConfusionMatrix(RandomForestClassifier())
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
oz = ROCAUC(RandomForestClassifier())
oz.fit(X_train, y_train)
oz.score(X_test, y_test)
oz.show()

In [ ]:

X = df2.loc[:, ~df2.columns.isin(['closed_ind','instname','unitid'])]
y = df2.loc[:, 'closed_ind']

# Specify the target classes
classes = ["open", "closed"]

# Create the training and test data
tscv = TimeSeriesSplit()
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Instantiate the classification model and visualizer
model = RandomForestClassifier()
visualizer = ClassificationReport(model, classes=classes, support=True)

visualizer.fit(X_train, y_train)        # Fit the visualizer and the model
visualizer.score(X_test, y_test)        # Evaluate the model on the test data
visualizer.show()                       # Finalize and show the figure

# Naive Bayes 

In [ ]:
cross_val_score(gnb, X, y, cv=10, n_jobs=1)

In [ ]:
cross_val_predict(gnb, X, y,cv=5, n_jobs=1)

In [ ]:
X = df2.loc[:, ~df2.columns.isin(['closed_ind','instname','unitid'])]
y = df2.loc[:, 'closed_ind']

# Specify the target classes
classes = ["open", "closed"]

# Create the training and test data
tscv = TimeSeriesSplit()
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Instantiate the classification model and visualizer
model = RandomForestClassifier()
visualizer = ClassificationReport(model, classes=classes, support=True)

visualizer.fit(X_train, y_train)        # Fit the visualizer and the model
visualizer.score(X_test, y_test)        # Evaluate the model on the test data
visualizer.show()                       # Finalize and show the figure

# K-Nearest Neighbors

In [ ]:
cross_val_score(knn, X, y, cv=10)

In [ ]:
cross_val_predict(knn, X, y,cv=10)

In [ ]:
X = df2.loc[:, ~df2.columns.isin(['closed_ind','instname','unitid'])]
y = df2.loc[:, 'closed_ind']

# Specify the target classes
classes = ["open", "closed"]

# Create the training and test data
tscv = TimeSeriesSplit()
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

# Instantiate the classification model and visualizer
model = GaussianNB()
visualizer = ClassificationReport(model, classes=classes, support=True)

visualizer.fit(X_train, y_train)        # Fit the visualizer and the model
visualizer.score(X_test, y_test)        # Evaluate the model on the test data
visualizer.show()                       # Finalize and show the figure

In [ ]:
# Load the classification data set
X = df2.loc[:, ~df2.columns.isin(['closed_ind','instname','unitid'])]
y = df2.loc[:, 'closed_ind']
model = RandomForestClassifier(n_estimators=10)
viz = FeatureImportances(model)
viz.fit(X, y)
viz.show()
